# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

For this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [13]:
## Connect to DB server on AWS
import mysql.connector 
import config
cnx = mysql.connector.connect(
    host = config.my_info['host'],
    user = config.my_info['user'],
    passwd = config.my_info['passwd'],
    database='Yelp_Business',
    port = config.my_info['port']
)
cursor = cnx.cursor()

# import config
# import mysql.connector
# cnx = mysql.connector.connect(
#                             host=config.mycred['host'],
#                             user=config.mycred['user'],
#                             password=config.mycred['password'],
#                             port=config.mycred['port'],
#                             database='Yelp_Business'

# )

# cursor = cnx.cursor()

In [12]:
## Create new DB
cursor.execute('''CREATE DATABASE Yelp_Business;
''')

In [14]:
# Create a table for the Businesses

# Create a table for the Businesses

#Columns: Name (string), Location(string), Review_Count (INT), Rating (FLoat), Price(String), Business_ID(string) (Primary Key)

cursor.execute('''CREATE TABLE Businesses (
                                Name varchar(255),
                                Location varchar(255),
                                Review_Count int,
                                Rating float,
                                Price varchar(255),
                                Business_ID varchar(255) NOT NULL,
                                PRIMARY KEY (Business_ID)
)

;

'''

)

In [15]:
# Create a table for the reviews

# Create a table for the reviews

#columns: Business ID(Foreign Key), Review Text(string), Review Date(String), Review ID (string)

cursor.execute('''CREATE TABLE Business_Reviews (
                                Review_Text varchar(255),
                                Review_Date varchar(255),
                                Review_ID varchar(255),
                                Business_ID varchar(255) NOT NULL,
                                FOREIGN KEY (Business_ID)
                                    REFERENCES Businesses (Business_ID))

;

'''

)

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [24]:
# Write a function to make a call to the yelp API

import requests

url = 'https://api.yelp.com/v3/businesses/search'
api_key = 'tmHkwzvKJnGUld8d7bqAN88EE15C3M0h-uD5Ku7ujqSRktBnJ4YIrs9Uc-1mvaUv81UBOFSteqdR1MXq4MtUsS6NylOgcV4-3BDrjPHhqgC3CJCYHZBMuPeKaX8oXnYx'

headers = {"Authorization": "Bearer {}".format(api_key),
}
 
term = 'chinese restaurant'

location = 'New York City'

limit = 50

offset = 50

url_params = {'term': 'chinese restaurant'.replace(' ', '+'),
              'location': 'New York City'.replace(' ', '+'),
              'limit': 50,
              'offset': 50
}

def call_yelp_data(url,headers, url_params):
    response = requests.get(url, headers=headers, params=url_params )
    data = response.json()
    business_data = data['businesses']
    return business_data

In [25]:
# business_data = call_yelp_data(url,headers, url_params) 
# why doesn't this work

In [26]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def parse_data(business_data):
    business_tuple_list = []
    for business in business_data:
        business_rev = business.get('price', 'No Price Listed')
        business_tuple = (business['name'],
                            business['location']['city'],
                            business['review_count'],
                            business['rating'],
                            business_rev,
                            business['id'])

        business_tuple_list.append(business_tuple)
    return business_tuple_list

In [27]:
insert_statement = ('''INSERT INTO Businesses (Name, Location, Review_Count, Rating, Price, Business_ID)
                        VALUES (%s, %s, %s, %s, %s, %s);
'''
)


def db_insert(cnx, cursor, parsed_results):
    cursor.executemany(insert_statement, parsed_results)
    cnx.commit()


## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [28]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
cursor.execute('''SELECT Business_ID FROM Businesses;
''')

In [29]:
results = cursor.fetchall()

In [30]:
# write a function that takes a business id 
# and makes a call to the API for reivews
list_of_business_ids = [item[0] for item in results]

In [31]:
review_url = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [32]:
#function to get reviews for one business ID which returns one dictionary of reviews
def one_business_review_call(business_id):
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    response = requests.get(url, headers=headers)
    if not response.ok:
        print(f"Response failed for {business_id}")
        print("response.text:", response.text)
        return list()
    data = response.json()
    return data['reviews']

In [33]:
#parse one business review set into a tuple. this returns one review tuple
def parse_business_review(review, biz_id):
    review_tuple = (review['text'],
                    review['time_created'],
                    review['id'],
                    biz_id)
    return review_tuple

In [34]:
#create an insert statement to insert review tuple into database
review_insert_statement = ('''INSERT INTO Yelp_Business.Business_Reviews (Review_Text,
                                Review_Date,
                                Review_ID,
                                Business_ID)
                                
                                VALUES (%s, %s, %s, %s);
'''
)

In [42]:
#create function to insert reviews for a business

def db_insert(cnx, cursor, review_tuple):
    # your code to insert and commit the results
    cursor.execute(review_insert_statement, review_tuple)
    cnx.commit()  

In [43]:
#call, parse, and insert tuples for each business ID

def call_parse_insert(cnx,cursor, list_of_business_ids):
    for biz_id in list_of_business_ids[212:]:
        print(f"fetching and inserting business {biz_id}")
        business_reviews = one_business_review_call(biz_id)
        for review in business_reviews:
            review_tuple = parse_business_review(review, biz_id)
            db_insert(cnx, cursor, review_tuple)

In [ ]:
# call_parse_insert(cnx,cursor, list_of_business_ids)

In [44]:
db_insert(cnx, cursor, review_tuple)

NameError: name 'review_tuple' is not defined

In [39]:
#variable for one business reviews

review_ids[0][0]

NameError: name 'review_ids' is not defined

In [41]:
# write a function that takes a business id 
# and makes a call to the API for reivews

def parse_reviews(business_reviews):
    review_tuple_list = []
    for review in business_reviews:
        review_tuple = (review['text'],
                        review['time_created'],
                        review['id'],
                        
                        
        )

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    response = requests.get(url, headers=headers, params=url_params )
    data = response.json()
    business_data = data['businesses']
    return business_data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    business_tuple_list = []
    for business in results:
        business_rev = business.get('price', 'No Price Listed')
        business_tuple = (business['name'],
                            business['location']['city'],
                            business['review_count'],
                            business['rating'],
                            business_rev,
                            business['id'])

        business_tuple_list.append(business_tuple)
    return business_tuple_list

In [ ]:
def db_insert(cnx, cursor, parsed_results):
    # your code to insert and commit the results
    cursor.executemany(insert_statement, parsed_results)
    cnx.commit()
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0
parsed_final = []
#set up a while loop to go through and grab the result 
while cur  < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    #add list of tuples to a final list
    parsed_final.extend(parsed_results)
    
    # use your function to insert your parsed results into the db
#     db_insert(cnx, cursor, parsed_results)
#     increment the counter by 50 to move on to the next results
    cur += 50

In [ ]:
# parsed_final
# db_insert(cnx, cursor, parsed_final)
cnx.close()